In [20]:
import datetime
import pandas as pd
from sqlalchemy import func, case, and_, desc, or_
from utils.finalyca_store import get_finalyca_scoped_session
from utils.utils import print_query
from fin_models.transaction_models import *
from fin_models.masters_models import *
from dateutil.relativedelta import relativedelta
import numpy as np

In [21]:
db_session = get_finalyca_scoped_session(False)

In [26]:
fundstock_query = db_session.query(
    FundStocks.HoldingSecurity_Name, 
    FundStocks.ISIN_Code, 
    
    func.sum(case((FundStocks.Product_Id == 1, 1), else_=0)).label('MF'),
    func.sum(case((FundStocks.Product_Id == 2, 1), else_=0)).label('ULIP'), 
    func.sum(case((FundStocks.Product_Id == 4, 1), else_=0)).label('PMS'),
    func.sum(case((FundStocks.Product_Id == 5, 1), else_=0)).label('AIF'),
    func.count(FundStocks.Product_Id).label('Total') ,
    
    func.sum(case((and_(FundStocks.Product_Id == 1, FundStocks.IncreaseExposure == 1), 1), else_=0)).label('IncreaseExposure_MF'),
    func.sum(case((and_(FundStocks.Product_Id == 2, FundStocks.IncreaseExposure == 1), 1), else_=0)).label('IncreaseExposure_ULIP'),
    func.sum(case((and_(FundStocks.Product_Id == 4, FundStocks.IncreaseExposure == 1), 1), else_=0)).label('IncreaseExposure_PMS'),
    func.sum(case((and_(FundStocks.Product_Id == 5, FundStocks.IncreaseExposure == 1), 1), else_=0)).label('IncreaseExposure_AIF'),
    func.sum(case((FundStocks.IncreaseExposure == 1, 1), else_=0)).label('IncreaseExposure_Total'),
    
    func.sum(case((and_(FundStocks.Product_Id == 1, FundStocks.DecreaseExposure == 1), 1), else_=0)).label('DecreaseExposure_MF'), 
    func.sum(case((and_(FundStocks.Product_Id == 2, FundStocks.DecreaseExposure == 1), 1), else_=0)).label('DecreaseExposure_ULIP'), 
    func.sum(case((and_(FundStocks.Product_Id == 4, FundStocks.DecreaseExposure == 1), 1), else_=0)).label('DecreaseExposure_PMS'), 
    func.sum(case((and_(FundStocks.Product_Id == 5, FundStocks.DecreaseExposure == 1), 1), else_=0)).label('DecreaseExposure_AIF'),
    func.sum(case((FundStocks.DecreaseExposure == 1, 1), else_=0)).label('DecreaseExposure_Total'),
    
    func.sum(case((and_(FundStocks.Product_Id == 1, FundStocks.NewStockForFund == 1), 1), else_=0)).label('NewStockForFund_MF'),
    func.sum(case((and_(FundStocks.Product_Id == 2, FundStocks.NewStockForFund == 1), 1), else_=0)).label('NewStockForFund_ULIP'), 
    func.sum(case((and_(FundStocks.Product_Id == 4, FundStocks.NewStockForFund == 1), 1), else_=0)).label('NewStockForFund_PMS'), 
    func.sum(case((and_(FundStocks.Product_Id == 5, FundStocks.NewStockForFund == 1), 1), else_=0)).label('NewStockForFund_AIF'),
    func.sum(case((FundStocks.NewStockForFund == 1, 1), else_=0)).label('NewStockForFund_Total'),
    
    func.sum(case((and_(FundStocks.Product_Id == 1, FundStocks.ExitStockForFund == 1), 1), else_=0)).label('ExitStockForFund_MF'),
    func.sum(case((and_(FundStocks.Product_Id == 2, FundStocks.ExitStockForFund == 1), 1), else_=0)).label('ExitStockForFund_ULIP'),
    func.sum(case((and_(FundStocks.Product_Id == 4, FundStocks.ExitStockForFund == 1), 1), else_=0)).label('ExitStockForFund_PMS'),
    func.sum(case((and_(FundStocks.Product_Id == 5, FundStocks.ExitStockForFund == 1), 1), else_=0)).label('ExitStockForFund_AIF'), 
    func.sum(case((FundStocks.ExitStockForFund == 1, 1), else_=0)).label('ExitStockForFund_Total'), 

    FundStocks.HoldingSecurity_Id,     
    FundStocks.MarketCap,
    FundStocks.Equity_Style,
    FundStocks.Sector_Code
).group_by(
    FundStocks.HoldingSecurity_Id,
    FundStocks.ISIN_Code, 
    FundStocks.HoldingSecurity_Name, 
    FundStocks.MarketCap, 
    FundStocks.Equity_Style, 
    FundStocks.Sector_Code
).filter(FundStocks.ISIN_Code.like("INE%")).filter(or_(FundStocks.InstrumentType=='Equity', FundStocks.Asset_Class=='Equity'))

fundstock_dt = fundstock_query.order_by(desc('Total')).all()

In [27]:
df = pd.DataFrame(fundstock_dt)

In [28]:
df

,HoldingSecurity_Name,ISIN_Code,MF,ULIP,PMS,AIF,Total,IncreaseExposure_MF,IncreaseExposure_ULIP,IncreaseExposure_PMS,...,NewStockForFund_Total,ExitStockForFund_MF,ExitStockForFund_ULIP,ExitStockForFund_PMS,ExitStockForFund_AIF,ExitStockForFund_Total,HoldingSecurity_Id,MarketCap,Equity_Style,Sector_Code
0,ICICI BANK LTD.,INE090A01021,594,412,120,24,1150,439,131,89,...,85,0,0,0,0,0,1400,Large Cap,Growth,8
1,HDFC BANK LTD.,INE040A01034,557,482,78,14,1131,147,258,8,...,85,0,0,0,0,0,6892,Large Cap,Growth,8
2,INFOSYS LTD.,INE009A01021,544,496,72,13,1125,116,315,16,...,93,0,0,0,0,0,745,Large Cap,Growth,12
3,RELIANCE INDUSTRIES LTD.,INE002A01018,511,506,64,14,1095,74,262,6,...,96,0,0,0,0,0,703,Large Cap,Growth,7
4,LARSEN & TOUBRO LTD.,INE018A01030,467,451,50,8,976,324,151,36,...,90,0,0,0,0,0,807,Large Cap,Growth,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,HEXAWARE TECHNOLOGIES LTD.,INE093A01033,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1493,Mid Cap,Value,12
1151,NUCLEUS SOFTWARE EXPORTS LTD.,INE096B01018,0,0,1,0,1,0,0,1,...,0,0,0,0,0,0,1534,Small Cap,Growth,12
1152,SIYARAM SILK MILLS LTD.,INE076B01028,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1315,Small Cap,Growth,13
1153,POWER MECH PROJECTS LTD,INE211R01019,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,2389,Small Cap,Growth,15


In [33]:
df.loc[df["HoldingSecurity_Name"] == 'PB FINTECH LTD.'] 

,HoldingSecurity_Name,ISIN_Code,MF,ULIP,PMS,AIF,Total,IncreaseExposure_MF,IncreaseExposure_ULIP,IncreaseExposure_PMS,...,NewStockForFund_Total,ExitStockForFund_MF,ExitStockForFund_ULIP,ExitStockForFund_PMS,ExitStockForFund_AIF,ExitStockForFund_Total,HoldingSecurity_Id,MarketCap,Equity_Style,Sector_Code
156,PB FINTECH LTD.,INE417T01026,66,34,2,0,102,0,0,0,...,16,0,0,0,0,0,49313,Mid Cap,-,11
